In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant AC data

### Reading the files and extracting relevant information:

In [5]:
plant = "AC"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  AC ['INSTRUÇÕES', 'Clínquer', '45', '33']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('AC', '45', array(['CPIIZ40'], dtype=object)),
 ('AC', '33', array(['CPIV32'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIIZ40': 1, 'CPIV32': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [13]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [14]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    .str.replace("D-G", "")
    .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [15]:
NUMERIC_COLUMNS = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string 

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [16]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [17]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Specific Gravity,100.000000
#400,100.000000
TiO2,100.000000
Initial setting time,89.819588
CS1,10.309278
K2O,3.221649
Na2O,3.221649
MgO,3.221649
CS28,1.804124
Al2O3,0.257732


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Specific Gravity</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>#400</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>TiO2</td>
        <td>100.00</td>
    </tr>  
    <tr>
        <td>Initial setting time</td>
        <td>89.82</td>
    </tr>
</table>

In [18]:
FEATRUES_TO_DROP = ["Specific Gravity", "#400", "TiO2", "Initial setting time"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Removing features with many zeros when applicable

In [19]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for AC
#200,0.644330
#325,0.128866
CaO,0.000000
MgO,0.000000
CS7,0.000000
CS3,0.000000
CS1,0.000000
Final setting time,0.000000
Blaine,0.000000
Insoluble Residue,0.000000


<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [20]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [21]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [22]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

### Dropping duplicated rows

In [23]:
df.shape

(762, 20)

<IPython.core.display.Javascript object>

In [24]:
df = df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [25]:
df = df.loc[df.select_dtypes("number").drop_duplicates().index].reset_index(drop=True)

chemical_vars = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
]

df = df.loc[
    df.select_dtypes("number").drop_duplicates(subset=chemical_vars).index
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [26]:
df.shape

(761, 20)

<IPython.core.display.Javascript object>

### Dropping rows with negative values

In [27]:
df[df.drop(["Date"], axis=1).duplicated()].drop("Date", axis=1).select_dtypes(
    include="number"
).apply(lambda x: x < 0).sum().sort_values(ascending=False)

CaO                   0.0
MgO                   0.0
CS7                   0.0
CS3                   0.0
CS1                   0.0
Final setting time    0.0
#325                  0.0
#200                  0.0
Blaine                0.0
Insoluble Residue     0.0
Loss on Ignition      0.0
Fe2O3                 0.0
K2O                   0.0
SO3                   0.0
SiO2                  0.0
Al2O3                 0.0
Na2O                  0.0
CS28                  0.0
dtype: float64

<IPython.core.display.Javascript object>

### Removing outliers

In [28]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,761,2022-05-13 01:00:33.114322944,2021-01-04 00:00:00,2022-06-13 00:00:00,2023-07-13 09:36:00,2023-07-21 00:00:00,NaN
CaO,760.0,45.595723,36.43,41.84,55.8987,57.77,6.392014
MgO,750.0,5.659103,4.14,5.37,7.6153,8.57,0.869773
Na2O,750.0,0.32613,0.02,0.3,0.47351,7.54,0.412847
Al2O3,760.0,8.552595,4.57,9.845,11.12902,11.43,2.225734
SiO2,760.0,28.338934,19.37,31.375,34.1964,34.86,5.057246
SO3,760.0,2.423671,1.84,2.28,3.16997,3.27,0.326237
K2O,750.0,1.467415,0.96,1.55,1.76,1.836,0.203727
Fe2O3,760.0,3.736117,2.73,4.01,4.39764,4.522,0.518397
Loss on Ignition,760.0,3.341421,2.01,3.32,4.37,5.4,0.354784


<IPython.core.display.Javascript object>

In [29]:
df = df.reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["#200"].eq(df["#200"].max())].reset_index(drop=True)
df = df[~df["#200"].eq(df["#200"].max())].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [30]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,755,2022-05-15 09:30:16.688741632,2021-01-04 00:00:00,2022-06-17 00:00:00,2023-07-13 11:02:24,2023-07-21 00:00:00,NaN
CaO,754.0,45.631591,36.43,41.865,55.9029,57.77,6.404562
MgO,744.0,5.664015,4.14,5.38,7.6171,8.57,0.871326
Na2O,744.0,0.297416,0.02,0.3,0.46,0.519,0.085128
Al2O3,754.0,8.53865,4.57,9.835,11.13034,11.43,2.228955
SiO2,754.0,28.310623,19.37,31.355,34.1988,34.86,5.06722
SO3,754.0,2.424708,1.84,2.28,3.17099,3.27,0.32726
K2O,744.0,1.466279,0.96,1.55,1.76,1.836,0.204121
Fe2O3,754.0,3.73281,2.73,4.005,4.39788,4.522,0.519047
Loss on Ignition,754.0,3.33935,2.01,3.32,4.3117,5.4,0.352666


<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [31]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

In [32]:
df.shape

(749, 20)

<IPython.core.display.Javascript object>

In [33]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
CS1,8.811749
MgO,1.468625
Na2O,1.468625
K2O,1.468625
Insoluble Residue,0.133511
Al2O3,0.133511
SiO2,0.133511
SO3,0.133511
Fe2O3,0.133511
Loss on Ignition,0.133511


<IPython.core.display.Javascript object>

In [34]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,749,2022-05-15 00:49:59.198931968,2021-01-04 00:00:00,2021-10-14 00:00:00,2022-06-17 00:00:00,2022-12-13 00:00:00,2023-07-21 00:00:00,NaN
CaO,748.0,45.594973,36.43,40.6475,41.84,53.405751,57.77,6.396332
MgO,738.0,5.662666,4.14,4.93,5.375,6.38,8.57,0.872268
Na2O,738.0,0.297965,0.02,0.25,0.3,0.36,0.519,0.084944
Al2O3,748.0,8.548091,4.57,5.8175,9.84,10.3625,11.43,2.228038
SiO2,748.0,28.331951,19.37,22.09,31.365,32.3725,34.86,5.063293
SO3,748.0,2.422753,1.84,2.17,2.278,2.7425,3.27,0.327216
K2O,738.0,1.467237,0.96,1.25,1.55,1.63,1.836,0.203779
Fe2O3,748.0,3.735439,2.73,3.12875,4.01,4.1525,4.522,0.518945
Loss on Ignition,748.0,3.340027,2.01,3.12,3.32,3.55,5.4,0.353319


<IPython.core.display.Javascript object>

In [35]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,749,NaN,NaN,NaN,2022-05-15 00:49:59.198931968,2021-01-04 00:00:00,2021-10-14 00:00:00,2022-06-17 00:00:00,2022-12-13 00:00:00,2023-07-21 00:00:00
Cement_Type,749,2,CP IV-32,477,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 749 entries, 0 to 754
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                749 non-null    datetime64[ns]
 1   CaO                 748 non-null    float64       
 2   MgO                 738 non-null    float64       
 3   Na2O                738 non-null    float64       
 4   Al2O3               748 non-null    float64       
 5   SiO2                748 non-null    float64       
 6   SO3                 748 non-null    float64       
 7   K2O                 738 non-null    float64       
 8   Fe2O3               748 non-null    float64       
 9   Loss on Ignition    748 non-null    float64       
 10  Insoluble Residue   748 non-null    float64       
 11  Blaine              749 non-null    float64       
 12  #200                749 non-null    float64       
 13  #325                749 non-null    float64       
 14 

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [37]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [38]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [39]:
df.to_csv("../../../../data/interim/209/ac.csv", index=False)

<IPython.core.display.Javascript object>